<img src="uni.png"
     alt="Universidade Logo" />
<br>
<br>
# Aprendizagem Automática - 2 Trabalho 

##### Realizado por : 
- Rúben Wilson nº 39837 
- Ludgero Teixeira nº 41348

#### <ins>Objetivo do trabalho</ins>: 

Utilizando informação do histórico académico dos alunos(programa, ECTS matriculados e concluídos e notas médias),construir um modelo preditivo que responda à pergunta:"quais os alunos em risco de abandonar os estudos?"

## Introdução
Sendo o objetivo deste trabalho obter um modelo preditivo que responda de uma forma eficiente à pergunta <i>"quais os alunos em risco de abandonar os estudos?"</i>, começamos então por analisar as várias possibilidades de algoritmos a utilizar e quais seriam aqueles que iriam ao encontro do tipo de dados disponíveis e ao problema a resolver. Após esta análise avançamos com os algoritmos <b>RandomForest</b> e <b>LogisticRegression</b>.

## 1º Modelo: Logistic Regression
<br>

### Imports

Inicialmente começamos por realizar os imports necessários para a construção do nosso modelo.

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

### Tratamento de Dados

Decidimos remover o atributo "Program" porque no nosso ponto de vista o tipo de curso a que cada aluno pertence não tem uma ligação direta para a criação de um modelo de previsão no âmbito de prever que alunos estão em risco de desistir.

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X = df.iloc[:, df.columns != 'Failure']
del X['Program'] 
y = df['Failure']
teste = test.iloc[:, test.columns != 'Program']

### Train_test_split

Usamos o método <b>train_test_split</b> para fazer a divisão do conjunto de dados na seguinte forma:
- <b>Conjunto Dados de Treino</b>
- <b>Conjunto Dados de Validação</b>

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

### Pipelines 

São uma forma de executar vários processos na ordem em que estão listados.
O objetivo do pipeline é reunir várias etapas que podem ser validadas em conjunto ao definir parâmetros diferentes.

Neste Pipeline os parâmetros escolhidos são foram:
 - <b>StandardScaler():</b> Normaliza os atributos removendo a média e escala para variação unitária.
 - <b>LogisticRegression():</b> A regressão logística é uma técnica estatística que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias.

In [4]:
logpipe = Pipeline([('scale', StandardScaler()),('logr',LogisticRegression())])

### Param_grid

Irá conter o conjunto de parâmetros do <i><b>Logistic Regression</b></i>, sendo estes os seguintes:
- <b>C:</b> Inverso da força de regularização -> <b>10.01</b>
- <b>solver:</b> Algoritmo a ser usado no problema de otimização. -> <b>lbfgs</b>
- <b>multi_class:</b> um problema binário é adequado para cada label -> <b>ovr</b>
- <b>penalty:</b> usado para especificar a norma usada na penalização -> <b>L2</b>

Para chegar a estes valores usamos a seguinte estrutura:
<br>
`param_grid = [
    {'logr__C':np.arange(0.01,100,10),
     'logr__penalty': ["l1","l2"]}
]`

In [5]:
param_grid = {
    'logr__C':[10.01],
    'logr__solver':['lbfgs'],
    'logr__multi_class':['ovr'],
    'logr__penalty':["l2"]}

### GridSearchCV & Fit

Usamos o <b>GridSearchCV</b> para realizar uma pesquisa exaustiva sobre valores de parâmetros especificados(param_grid) para um estimador(<b>LogisticRegression()</b>).
Este usa os seguintes parâmetros:
- <b>logregpipe:</b> Pipeline defina acima
- <b>param_grid:</b> Conjunto de parâmetros a utilizar
- <b>cv:</b> Determina a estratégia de divisão para a validação cruzada -> <b>10</b>

Após efetuarmos o <b>GridSearchCV</b> precedemos ao <i><b>logpipe_cv.fit</b></i> atribuindo-o à variável <b>best_clf</b>

In [6]:
logpipe_cv = GridSearchCV(logpipe , param_grid , cv = 10, return_train_score = True)
best_clf = logpipe_cv.fit(X_train,y_train)

{'logr__C': 10.01, 'logr__multi_class': 'ovr', 'logr__penalty': 'l2', 'logr__solver': 'lbfgs'}


### Print da Exatidão & Criação do conjunto de previsão

Primeiro faz print da <b>exatidão(score)</b>, onde usando o nosso <b>modelo</b> e o <b>conjunto de validação (x_test, y_test)</b>.
<br>
O conjunto de previsão após ser calculado é guardado num ficheiro <i><b>LRprevisao.csv</b></i> numa coluna chamada previsões, sendo que para cada <i><b>id</b></i> do ficheiro <i><b>test</b></i> ele gera essa mesma previsão.

In [7]:
print('A exatidão é de:',best_clf.score(X_val, y_val))
prediction = pd.DataFrame(best_clf.predict(teste), columns=['previsoes']).to_csv('LRprevisoes.csv')

A exatidão é de: 0.9437229437229437


### Observações Finais deste modelo

<b>Logistic Regression</b> como primeiro modelo de previsão testado apresentou logo bons resultados <i>(melhores do que estavamos à espera)</i> para o tipo de problema em mãos.
Os parâmetros utilizados para a construção do modelo de previsão indicam ser os mais adequados, pois após várias tentativas entre escolha de parâmetros e de valores dos mesmos, aqueles que apresentaram melhores resultados mantiveram-se, sendo que a tentativa de arranjar outras formas de aumentar a exatidão do modelo não tiveram qualquer sucesso.
Sendo que o valor de exatidão para o conjunto de teste flutua entre os <b>94%</b> e os <b>95%</b> é possível afirmar que foi feita uma boa previsão, não exedendo os valores ao ponto de haver <ins>overfitting</ins>.

## 2º Modelo: Random Forest

### Imports

Inicialmente começamos por realizar os imports necessários para a construção do nosso modelo.

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd

### Tratamento de Dados

De forma a encontrar os dados realmente necessários para o problema em causa e permitir construir o nosso modelo de uma melhor forma, acabamos por decidir para este modelo adaptar uma perspetiva diferente em relação à do modelo <b>LogisticRegression</b>.<br>
Assim sendo, decidimos retirar inicialmente a coluna <i><b>"Program"</b></i> pela mesma razão que mencionamos no modelo do anterior e seguidamente todas as colunas denomidas por <i><b>"..._grade"</b></i> , isto porquê?
<br>Porque, por um lado a média de cada aluno de cada semestre acaba por não ser uma feature decisiva para alguém desistir de um curso ou não, e se essa mesma média acabar por estar refletida já nos <b>ECTS</b> completados.<br>
Por outro, imaginando um aluno inscrito a 36 ECTS e completa 2 ECTS, sendo que nesses dois obteve uma classificação de 18 valores a média que iria aparecer na coluna para esse mesmo aluno seria 18, logo acaba por ser uma média <b>"Falsa"</b> que nos pode levar a erros...

- <b>train2.csv:</b> Ficheiro dados treino alterado segundo as razões mencionadas acima
- <b>test2.csv:</b> Ficheiro dados teste alterado segundo as razões mencionadas acima


In [9]:
df = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

X = df.iloc[:, df.columns != 'Failure']

y = df['Failure']

### Train_test_split

Usamos o método <b>train_test_split</b> para fazer a divisão do conjunto de dados na seguinte forma:
- <b>Conjunto Dados de Treino</b>
- <b>Conjunto Dados de Validação</b>

In [10]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.28,random_state=0)

### Pipelines 

São uma forma de executar vários processos na ordem em que estão listados.
O objetivo do pipeline é reunir várias etapas que podem ser validadas em conjunto ao definir parâmetros diferentes.
No modelo abaixo, definimos o <i><b>Random Forest Classifier</b></i>.

In [11]:
pipe = Pipeline([('classifier', RandomForestClassifier())]) 

### Param_grid

Irá conter o conjunto de parâmetros do <i><b>RandomForest</b></i>, sendo estes os seguintes:
- <b>n_estimators:</b> O número de árvores -> <b>80</b>
- <b>criterion:</b> Critério usado para medir a qualidade de uma divisão -> <b>entropy</b>
- <b>max_features:</b> O número de features a serem considerados ao procurar a melhor divisão -> <b>18</b>

Para chegar a estes valores usamos a seguinte estrutura:
<br>
`param_grid = [
    {'classifier__n_estimators':  list(range(10, 101, 10)),
     'classifier__criterion': ['entropy','gini'],
     'classifier__max_features': list(range(1,31))}
]`
<br>
Sendo que depois de efetuarmos o <i><b>GridSearchCV</b></i> e o método <i><b>fit</b></i> usamos o `print(best_clf.best_params_)` para encontrar esses mesmo valores!

In [12]:
param_grid = [
    {'classifier__n_estimators': [80],  
     'classifier__criterion': ['entropy'],
     'classifier__max_features': [18]}]

### GridSearchCV & Fit

Usamos o <b>GridSearchCV</b> para realizar uma pesquisa exaustiva sobre valores de parâmetros especificados(param_grid) para um estimador(<b>RandomForestClassifier()</b>).
Este usa os seguintes parâmetros:
- <b>pipe:</b> Pipeline definida acima
- <b>param_grid:</b> Conjunto de parâmetros a utilizar
- <b>cv:</b> Determina a estratégia de divisão para a validação cruzada -> <b>5</b>

Após efetuarmos o <b>GridSearchCV</b> procedemos ao <i><b>clf.fit</b></i> atribuindo-o à variável <b>best_clf</b>

In [13]:
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
best_clf = clf.fit(x_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


### Print da Exatidão & Criação do conjunto de previsão

Primeiro faz print da <b>exatidão(score)</b>, onde usando o nosso <b>modelo</b> e o <b>conjunto de validação (x_test, y_test)</b>.
<br>
O conjunto de previsão após ser calculado é guardado num ficheiro <i><b>previsoes2.csv</b></i> numa coluna chamada previsoes, sendo que para cada <i><b>id</b></i> do ficheiro <i><b>test</b></i> ele gera essa mesma previsão.

In [14]:
print('A exatidão é de:',best_clf.score(x_val, y_val))

prediction = pd.DataFrame(best_clf.predict(test), columns=['previsoes']).to_csv('RFprevisoes.csv')

A exatidão é de: 0.9458413926499033


### Observações Finais deste modelo

Apresenta ser um modelo eficaz para este tipo de problema sendo que com esta parametrização obtemos valores de exatidão entre <b>94% - 96%</b>.
No entanto tentámos efetuar e encontrar outros parâmetros que nos permitissem obter valores de exatidão mais elevados mas essas mesmas tentivas não tiveram qualquer sucesso...
<br>Uma das causas possíveis para o não sucesso da respetiva melhoria será a forma como tratamos os dados inicialmente, possivelmente olhando, alterando e criando dados novos e mais eficientes como base nos dados iniciais, ou seja, <b>uma perspetiva diferente</b> do que aquela que decidimos traçar, muito possivelmente teriamos obtido resultados no que se trata na melhoria dos valores de exatidão.

## Bibliografias

- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html